In [1]:
"""Output Transactions

This notebook shows how LUSID uses synthetic transactions to fill in the gaps between user-instructed transactions and corporate actions.

Attributes
----------

"""

'Output Transactions\n\nThis notebook shows how LUSID uses synthetic transactions to fill in the gaps between user-instructed transactions and corporate actions.\n\nAttributes\n----------\n\n'

# Output Transactions

This notebook shows how LUSID uses synthetic transactions to fill in the gaps between user-instructed transactions and corporate actions.


## Setup LUSID




In [4]:
# Import system packages

# Import lusid specific packages
# These are the core lusid packages for interacting with the API via Python
import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response,
)
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame

import os
import pandas as pd

pd.set_option('display.max_columns', None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

## Load source data

Create a scope and portfolio code

In [5]:
scope = "output-transactions"
portfolio_code = "EQUITY-UK"

In [6]:
df = pd.read_csv("data/equity_transactions.csv")
df

,portfolio_code,portfolio_name,portfolio_base_currency,ticker,sedol,instrument_type,instrument_id,name,txn_id,txn_type,txn_trade_date,txn_settle_date,txn_units,txn_price,txn_consideration,currency,strategy,cash_transactions
0,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0002162385,SEDOL1,equity,EQ_1234,Aviva,trd_0001,Buy,02/01/2020,04/01/2020,120000,5,600000,GBP,ftse_tracker,NaN
1,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0002162385,SEDOL1,equity,EQ_1234,Aviva,trd_0002,Buy,02/01/2020,04/01/2020,12000,5,60000,GBP,ftse_tracker,NaN
2,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0000566504,SEDOL2,equity,EQ_1235,BHP,trd_0003,Buy,02/01/2020,04/01/2020,60000,18,1080000,GBP,ftse_tracker,NaN
3,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0000566504,SEDOL2,equity,EQ_1235,BHP,trd_0004,Buy,02/01/2020,04/01/2020,60000,18,1080000,GBP,ftse_tracker,NaN
4,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0031348658,SEDOL3,equity,EQ_1236,Barclays,trd_0005,Buy,02/01/2020,04/01/2020,150000,2,300000,GBP,ftse_tracker,NaN
5,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0031348658,SEDOL3,equity,EQ_1236,Barclays,trd_0006,Buy,02/01/2020,04/01/2020,150000,2,300000,GBP,ftse_tracker,NaN
6,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0007980591,SEDOL4,equity,EQ_1237,BP,trd_0007,Buy,02/01/2020,04/01/2020,100000,5,500000,GBP,ftse_tracker,NaN
7,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0007980591,SEDOL4,equity,EQ_1237,BP,trd_0008,Buy,02/01/2020,04/01/2020,100000,5,500000,GBP,ftse_tracker,NaN
8,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0005405286,SEDOL5,equity,EQ_1238,HSBC,trd_0009,Buy,02/01/2020,04/01/2020,20000,6,120000,GBP,ftse_tracker,NaN
9,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0005405286,SEDOL5,equity,EQ_1238,HSBC,trd_0010,Buy,02/01/2020,04/01/2020,20000,6,120000,GBP,ftse_tracker,NaN


## Load instruments

Create the instruments and add on ISIN and Sedol as identifiers

In [9]:
instrument_mapping = {
    "identifier_mapping": {
        "ClientInternal": "instrument_id",
        "Isin": "ticker",
        "Sedol": "sedol",
    },
    "required": {
        "name": "name"
    },
}

In [10]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=df,
    mapping_required=instrument_mapping["required"],
    mapping_optional={},
    file_type="instruments",
    identifier_mapping=instrument_mapping["identifier_mapping"],
)

succ, failed, errors = format_instruments_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,11,0,0


## Create portfolio

In [14]:
portfolio_mapping = {
    "required": {
        "code": "portfolio_code",
        "display_name": "portfolio_name",
        "base_currency": "$GBP",
    },
    "optional": {"created": "$2020-01-01T00:00:00+00:00"},
}

In [16]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=df,
    mapping_required=portfolio_mapping["required"],
    mapping_optional=portfolio_mapping["optional"],
    file_type="portfolios",
    sub_holding_keys=[],
)

succ, failed = format_portfolios_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,1,0,0


## Load in transactions

In [17]:
transaction_mapping = {
    "identifier_mapping": {
        "ClientInternal": "instrument_id",
    },
    "required": {
        "code": "portfolio_code",
        "transaction_id": "txn_id",
        "type": "txn_type",
        "transaction_price.price": "txn_price",
        "transaction_price.type": "$Price",
        "total_consideration.amount": "txn_consideration",
        "units": "txn_units",
        "transaction_date": "txn_trade_date",
        "total_consideration.currency": "portfolio_base_currency",
        "settlement_date": "txn_settle_date",        
    },
    "optional": {},
    "properties": [],
}

In [18]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=df,
    mapping_required=transaction_mapping["required"],
    mapping_optional=transaction_mapping["optional"],
    file_type="transactions",
    identifier_mapping=transaction_mapping["identifier_mapping"],
    property_columns=transaction_mapping["properties"],
    properties_scope=scope,
)

succ, failed = format_transactions_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(failed)}])

,success,failed,errors
0,1,0,0
